<img src="https://cursos.virtual.uniandes.edu.co/isis4219/wp-content/uploads/sites/162/2014/11/cropped-misisheader.png" ><br>
# Machine Learning Techniques - ISIS4219

Intersemestral 2020

### Integrantes:
* Juan José Villegas
* Juan Sebastián Méndez

De acuerdo a los conceptos vistos en clase desarrolle el siguiente ejercicio práctico:

 1. El Internet de las Cosas (IoT) está desempeñando un papel cada vez más importante en nuestras actividades cotidianas, al habilitar la conexión de los objetos que nos rodean con servicios digitales. 
 
 Últimamente, IoT se ha visto afectado por una variedad de botnets, los cuales intentan explotar vulnerabilidades en los protocolos de aplicación que interactúan directamente con los sistemas, provocando violaciones de seguridad. 
 
 Una solución a este tipo de ataques es identificar, de manera automática, eventos maliciosos contra los protocolos de comunicación. En este sentido, ***se desea desarrollar un sistema de detección de intrusos que permita una identificación temprana de estas amenazas***. Sin embargo, el Departamento de Seguridad quiere identificar que ataque específico se está ejecutando para poder implementar los protocolos correspondientes.

Como recomendación general, se espera que para la entrega del taller se aplique la estructura establecida por la metodología CRISP- DM, según corresponda con las caracteristicas de cada conjunto de datos.

# 1. Importando las librerias requeridas para el desarrollo del laboratorio

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.decomposition import TruncatedSVD

# 2. Perfilamiento y entendimiento de los datos

In [2]:
#Importando los datos
diccionario = pd.read_csv('./Punto 1/NUSW-NB15_diccionario.csv', encoding = "ISO-8859-1")
train = pd.read_csv('./Punto 1/UNSW_NB15_training-set.csv', encoding = "ISO-8859-1")
test = pd.read_csv('./Punto 1/UNSW_NB15_testing-set.csv', encoding = "ISO-8859-1")

In [3]:
#Resultado de ProfileReport se encuentra en profile_report.html
#ProfileReport(train)

In [4]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

<img src="./Imagenes/Correlaciones.png" >

## Notas:
* El label no es necesario porque lo que se quiere es predicir el tipo de ataque. Existe el tipo "normal" que es equivale al label 0. Columna se elimina.
* La columna de Id no es necesaria. Columna se elimina.
* Se observa de la tabla de correlación y del reporte del ProfileReport que existen variables altamente correlacionadas. Se eliminan aquellas columnas que manejan bytes (Ejemplo, dbytes).
* Hay que tranformar los atributos categóricos proto, service y state a números.
* Hay que escalar todas las variable ya que están muy dispersas

# 3. Limpieza y preparación de datos

La idea en este paso es la de remover aquellos datos que el algoritmo no va a aprender y que no es capás de predecir del conjunto de test. Se encuentran entonces, de las columnas categóricas, aquellas categorías que sólo se encuentran en el conjunto de test, y se eliminan. Igualmente, adicionamos aquellos datos que sólo se encuentran en el conjunto de train, en el de test, para que el one hot encoder nos arroje la misma forma.

In [5]:
def analyze_categorical_columns(train, test, features_to_drop):
    X_train = train.drop(features_to_drop, axis=1)
    X_test = test.drop(features_to_drop, axis=1)

    X_train_proto_categories = np.sort(X_train.proto.unique())
    X_test_proto_categories = np.sort(X_test.proto.unique())
    diff_proto = list(set(X_test_proto_categories)-set(X_train_proto_categories))
    diff_proto2 = list(set(X_train_proto_categories)-set(X_test_proto_categories))
    print('differences proto (in test only): ', diff_proto)
    print('differences proto (in train only): ', diff_proto2)

    X_train_service_categories = np.sort(X_train.service.unique())
    X_test_service_categories = np.sort(X_test.service.unique())
    diff_service = list(set(X_test_service_categories)-set(X_train_service_categories))
    diff_service2 = list(set(X_train_service_categories)-set(X_test_service_categories))
    print('differences service (in test only): ', diff_service)
    print('differences service (in train only): ', diff_service2)

    X_train_state_categories = np.sort(X_train.state.unique())
    X_test_state_categories = np.sort(X_test.state.unique())
    diff_state = list(set(X_test_state_categories)-set(X_train_state_categories))
    diff_state2 = list(set(X_train_state_categories)-set(X_test_state_categories))
    print('differences state (in test only): ', diff_state)
    print('differences state (in train only): ', diff_state2)

    return diff_proto, diff_service, diff_state

In [6]:
# Drop features that are not needed
# We should think of droping highly correlated features 
features_to_drop = ['label', 'attack_cat', 'ï»¿id', 'sbytes', 'dbytes', 'dloss', 'sloss']
diff_proto, diff_service, diff_state = analyze_categorical_columns(train, test, features_to_drop)

differences proto (in test only):  ['rtp', 'icmp']
differences proto (in train only):  []
differences service (in test only):  []
differences service (in train only):  []
differences state (in test only):  ['ECO', 'no', 'PAR', 'URN']
differences state (in train only):  ['ACC', 'CLO']


Del analisis anterior vemos que la columna service contiene las mismas categorias en ambos conjuntos de datos. Por el contrario, proto y state tiene categorías que sólo pertenecen a test, y éste último también tiene datos (ACC y CLO) que sólo existen en el de train. Hacemos las correspondientes adiciones y substracciones.

In [7]:
X_train, y_train = train.drop(features_to_drop, axis=1), train['attack_cat']

acc_df = train.loc[train['state'] == 'ACC']
clo_df = train.loc[train['state'] == 'CLO']
test = test.append(acc_df)
test = test.append(clo_df)

test = test[~test['proto'].isin(diff_proto)]
test = test[~test['service'].isin(diff_service)]
test = test[~test['state'].isin(diff_state)]

X_test, y_test = test.drop(features_to_drop, axis=1), test['attack_cat']

Ahora la idea es estandarizar los datos: Para esto utilizamos un pipeline con dos encoders. El primero es un MinMaxScaler que nos deja en la misma escala todas las columnas. El segundo es un oneHotEncoder que nos separa cada categoría en su propia columna.

In [8]:
num_preprocessor = make_pipeline(
    MinMaxScaler(feature_range=(0,1))
)

categories_preprocessor = make_pipeline(
    OneHotEncoder()
)

preprocessor = make_column_transformer(
    (num_preprocessor, ('dur','spkts','dpkts','rate','sttl','dttl','sload','dload','sinpkt','dinpkt','sjit', 'djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean', 'trans_depth', 'response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm', 'is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports')),
    (categories_preprocessor, ('proto', 'service', 'state')),
    remainder = 'passthrough'
)

In [9]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.fit_transform(X_test)
print('train shape: ',X_train_prepared.shape)
print('test shape: ',X_test_prepared.shape)

train shape:  (82332, 186)
test shape:  (175330, 186)


# 4. Modelamiento

In [10]:
model = SVC(kernel='rbf', class_weight='balanced', cache_size=5000, random_state=42)

**Sintonización de hiperparámetros**

In [11]:
param_grid = {'C': [1, 5, 10, 50],
             'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid,scoring='recall_micro', return_train_score=True,n_jobs=-1)
%time grid.fit(X_train_prepared, y_train)
print(grid.best_params_)

CPU times: user 7min 5s, sys: 51.6 s, total: 7min 56s
Wall time: 16h 54min 12s
{'C': 50, 'gamma': 0.005}


In [12]:
grid.best_score_

0.6409373808735145

# 5. Predicción

In [13]:
model = grid.best_estimator_

In [14]:
yfit = model.predict(X_test_prepared)

# 6. Validación

In [15]:
print(classification_report(y_test, yfit))

precision    recall  f1-score   support

      Analysis       0.02      0.01      0.02      2000
      Backdoor       0.06      0.04      0.05      1746
           DoS       0.33      0.75      0.46     12264
      Exploits       0.81      0.39      0.53     33394
       Fuzzers       0.53      0.56      0.55     18185
       Generic       0.99      0.98      0.99     40000
        Normal       0.95      0.83      0.88     55986
Reconnaissance       0.58      0.35      0.43     10492
     Shellcode       0.08      0.61      0.14      1133
         Worms       0.02      0.84      0.03       130

      accuracy                           0.70    175330
     macro avg       0.44      0.54      0.41    175330
  weighted avg       0.80      0.70      0.72    175330

